In [1]:
!pip install replay-rec implicit -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.4 MB/s eta 0:00:00
ERROR: pip's dependency reso

In [2]:
from collections import Counter
import random
import warnings
warnings.simplefilter(action='ignore')

import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds

from implicit.als import AlternatingLeastSquares
from replay.metrics import HitRate, NDCG, MAP, Experiment
from replay.preprocessing.converter import CSRConverter
from replay.preprocessing.filters import MinCountFilter, LowRatingFilter
from replay.splitters import TimeSplitter

In [3]:
random.seed(0)
np.random.seed(0)

In [4]:
USER_COL = 'user_id'
ITEM_COL = 'item_id'
RATING_COL = 'rating'
TIMESTAMP = 'timestamp'

#  Загрузка данных

In [5]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip

--2024-08-22 07:00:45--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  10.6MB/s    in 0.5s    

2024-08-22 07:00:46 (10.6 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [6]:
ratings = pd.read_csv('ml-1m/ratings.dat',
                      sep='::',
                      names=[USER_COL, ITEM_COL, RATING_COL, TIMESTAMP])
ratings.head()

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
ratings[TIMESTAMP] = pd.to_datetime(ratings[TIMESTAMP], unit='s')

Переведем рейтинги в float формат для корректной работы моделей.

In [8]:
ratings[RATING_COL] = ratings[RATING_COL].astype(float)

В датасете содержатся только пользователи с более чем 20 просмотренными фильмами. Отфильтруем фильмы, у которых менее 20 просмотров.

In [9]:
ratings = MinCountFilter(num_entries=20, groupby_column=ITEM_COL).transform(ratings)

In [10]:
print(f'Количество уникальных пользователей: {ratings[USER_COL].nunique()}')
print(f'Количество уникальных фильмов: {ratings[ITEM_COL].nunique()}')

Количество уникальных пользователей: 6040
Количество уникальных фильмов: 3043


## train-test split

Разделим данные по времени так, чтобы 20% взаимодействий оказалось в тестовой выборке. Исключим из тестовой выборки холодных пользователей и холодные объекты.

In [11]:
train, test = TimeSplitter(time_threshold=0.2,
                           query_column=USER_COL,
                           item_column=ITEM_COL,
                           drop_cold_users=True,
                           drop_cold_items=True).split(ratings)
train.shape, test.shape

((796392, 4), (103764, 4))

Поменяем индексы пользователей и фильмов так, чтобы в нумерации не было пропусков.

In [12]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [13]:
all_users = train[USER_COL].astype('category').cat.codes
all_items = train[ITEM_COL].astype('category').cat.codes

user_id2idx = dict(zip(train[USER_COL], all_users))
item_id2idx = dict(zip(train[ITEM_COL], all_items))

train[USER_COL] = train[USER_COL].map(user_id2idx)
train[ITEM_COL] = train[ITEM_COL].map(item_id2idx)
test[USER_COL] = test[USER_COL].map(user_id2idx)
test[ITEM_COL] = test[ITEM_COL].map(item_id2idx)

Будем оценивать качество только на фильмах, имеющих высокие рейтинги в тестовых данных.

In [14]:
test = LowRatingFilter(value=4, rating_column=RATING_COL).transform(test)
test.shape

(55988, 4)

# Models

Базовый класс для формирования разреженной матрицы рейтингов и получения предсказаний.

In [15]:
class BaseFactorizationModel:
    def __init__(
        self,
        random_state=0,
        user_col=USER_COL,
        item_col=ITEM_COL,
        rating_col=RATING_COL,
    ):
        self.random_state = np.random.RandomState(random_state)
        self.user_col = user_col
        self.item_col = item_col
        self.rating_col = rating_col

    def get_rating_matrix(self, data):
        return CSRConverter(
            first_dim_column=self.user_col,
            second_dim_column=self.item_col,
            data_column=self.rating_col,
        ).transform(data)

    def predict(self, scores, rating_matrix=None, filter_seen=True, k=10):
        if filter_seen:
            scores = np.multiply(
                scores,
                np.invert(rating_matrix.todense().astype(bool))
                )

        ind_part = np.argpartition(scores, -k + 1)[:, -k:].copy()
        scores_not_sorted = np.take_along_axis(scores, ind_part, axis=1)
        ind_sorted = np.argsort(scores_not_sorted, axis=1)
        scores_sorted = np.sort(scores_not_sorted, axis=1)
        indices = np.take_along_axis(ind_part, ind_sorted, axis=1)

        preds = pd.DataFrame({
            self.user_col: range(scores.shape[0]),
            self.item_col: np.flip(indices, axis=1).tolist(),
            self.rating_col: np.flip(scores_sorted, axis=1).tolist()
            }).explode([self.item_col, self.rating_col])

        return preds

# PureSVD

In [16]:
class SVD(BaseFactorizationModel):
    def __init__(
        self, random_state=0, user_col=USER_COL, item_col=ITEM_COL, n_factors=20
        ):
        super().__init__(random_state, user_col, item_col)
        self.n_factors = n_factors


    def fit(self, data):
        self.rating_matrix = self.get_rating_matrix(data)
        user_matrix, singular_values, item_matrix = svds(
            A=self.rating_matrix, k=self.n_factors
            )
        user_matrix = user_matrix * np.sqrt(singular_values)
        item_matrix = item_matrix.T * np.sqrt(singular_values)

        self.user_matrix = user_matrix
        self.item_matrix = item_matrix
        self.scores = user_matrix @ item_matrix.T

Обучим модель.

In [17]:
svd_model = SVD()
svd_model.fit(train)

Получим предсказания.

In [18]:
preds_svd = svd_model.predict(svd_model.scores, svd_model.rating_matrix)

In [19]:
preds_svd.head(10)

,user_id,item_id,rating
0,0,900,3.12136
0,0,886,2.788202
0,0,954,2.31215
0,0,1175,2.054553
0,0,463,2.032025
0,0,884,2.030257
0,0,2237,1.95366
0,0,2701,1.752146
0,0,1053,1.70703
0,0,690,1.681457


Вычислим метрики качества для top-10 предсказанных фильмов у каждого пользователя.

In [20]:
K = [10]
metrics = Experiment(
    [
        NDCG(K),
        MAP(K),
        HitRate(K),
    ],
    test,
    query_column=USER_COL,
    item_column=ITEM_COL,
)

In [21]:
metrics.add_result("SVD", preds_svd)
metrics.results

,NDCG@10,MAP@10,HitRate@10
SVD,0.233851,0.139523,0.673214


## iALS

Сформируем разреженную матрицу рейтингов.

In [22]:
base_model = BaseFactorizationModel()
rating_matrix = base_model.get_rating_matrix(train)

Обучим модель.

In [34]:
ials_model = AlternatingLeastSquares(factors=20,
                                     regularization=0.1,
                                     iterations=50,
                                     use_gpu=False)
ials_model.fit((rating_matrix).astype('double'))

  0%|          | 0/50 [00:00<?, ?it/s]

Получим обученные векторы пользователей и объектов и матрицу скоров для каждой пары пользователь-объект.



In [35]:
user_vecs = ials_model.user_factors
item_vecs = ials_model.item_factors
scores = user_vecs.dot(item_vecs.T)

Получим предсказания.

In [36]:
preds_ials = base_model.predict(scores, rating_matrix)

Оценим значения метрик.

In [37]:
metrics.add_result("iALS", preds_ials)
metrics.results

,NDCG@10,MAP@10,HitRate@10
SVD,0.233851,0.139523,0.673214
iALS,0.208768,0.120568,0.643750


##  Векторы пользователей c похожими и различными вкусами

In [38]:
ials_model.similar_users(userid=0)

(array([   0, 3692, 1084, 5305, 3356, 4398, 3479, 4628, 4912, 1816],
       dtype=int32),
 array([1.0000001 , 0.91342753, 0.88905513, 0.8780129 , 0.86771363,
        0.8632521 , 0.86283153, 0.8615592 , 0.85147566, 0.85124373],
       dtype=float32))

In [39]:
similar_user_ind = ials_model.similar_users(userid=0)[0][1]
similar_user_ind

3692

Сравним векторы похожих пользователей, полученные при обучении модели.

In [40]:
user_vecs[0]

array([-0.62860894,  1.3941152 ,  0.10064855,  0.03237688, -0.46190938,
       -0.5431889 ,  0.09382162,  0.66197056, -0.00831141,  0.8183079 ,
        1.762416  , -0.03051501, -0.44292903,  0.34098735,  0.4682741 ,
        0.99580294, -0.88575935,  0.96705574,  0.48450026,  0.319992  ],
      dtype=float32)

In [41]:
user_vecs[similar_user_ind]

array([-1.4004424e+00,  2.0445070e+00, -1.0367923e-01, -3.4918401e-01,
       -1.0586486e-01, -7.9619986e-01,  1.7912245e-01,  1.1136132e+00,
        3.1763053e-01,  1.3622116e+00,  2.4528048e+00, -7.3098838e-02,
        4.7298357e-01,  9.4139016e-01,  2.3870263e-04,  2.1087735e+00,
       -5.8262354e-01,  9.4402128e-01,  9.1207230e-01,  4.9066082e-01],
      dtype=float32)

Найдем наименее похожего пользователя.

In [42]:
dissimilar_user_ind = ials_model.similar_users(userid=0, N=train[ITEM_COL].nunique())[0][-1]
dissimilar_user_ind

936

In [43]:
user_vecs[dissimilar_user_ind]

array([-0.40155157, -0.31058863,  0.7733315 ,  0.18352339, -0.18971469,
        0.72035396,  0.32691592,  0.8258542 ,  0.70629716, -0.21478705,
        0.9113024 ,  0.22579816,  0.7856044 ,  0.13113108,  0.25329167,
       -0.3240278 , -0.15114535,  0.7249473 ,  0.18032338, -0.30015352],
      dtype=float32)

Посмотрим количество одинаковых фильмов в истории просмотров похожих пользователей.

In [44]:
user_items = set(train[train[USER_COL] == 0][ITEM_COL])
similar_user_items = set(train[train[USER_COL] == similar_user_ind][ITEM_COL])
dissimilar_user_items = set(train[train[USER_COL] == dissimilar_user_ind][ITEM_COL])

In [45]:
print(f'Пользователь с индексом 0 посмотрел {len(user_items)} фильма')
print(f'Похожий на него пользователь посмотрел {len(similar_user_items)} фильмов')
print(f'Количество фильмов, которые посмотрели оба пользователя: {len(user_items & similar_user_items)}')

Пользователь с индексом 0 посмотрел 64 фильма
Похожий на него пользователь посмотрел 157 фильмов
Количество фильмов, которые посмотрели оба пользователя: 33


In [46]:
print(f'Непохожий на пользователя с индексом 0 пользователь посмотрел {len(dissimilar_user_items)} фильма')
print(f'Количество фильмов, которые посмотрели оба пользователя: {len(user_items & dissimilar_user_items)}')

Непохожий на пользователя с индексом 0 пользователь посмотрел 52 фильма
Количество фильмов, которые посмотрели оба пользователя: 2


## Предсказание для одного пользователя

Загрузим данные о названиях фильмов.

In [47]:
movies = pd.read_csv('ml-1m/movies.dat', sep='::', encoding='latin-1', names=[ITEM_COL, 'title', 'genre'])
movies.head()

,item_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [48]:
movies[ITEM_COL] = movies[ITEM_COL].map(item_id2idx)

In [49]:
movies['genre'] = movies.pop('genre').str.split('|')

Получим список наиболее предпочтительных для пользователя жанров.

In [50]:
user_genre_list = sum(train[train[USER_COL] == 0].merge(movies, on=ITEM_COL)['genre'].tolist(), [])
user_genre_count = Counter(user_genre_list)
sorted(user_genre_count.items(), key=lambda item: item[1], reverse=True)

[('Sci-Fi', 37),
 ('Horror', 32),
 ('Action', 26),
 ('Adventure', 15),
 ('Thriller', 13),
 ('Comedy', 11),
 ('Drama', 9),
 ('Romance', 4),
 ('Fantasy', 4),
 ('Animation', 3),
 ('War', 3),
 ("Children's", 2),
 ('Crime', 2),
 ('Musical', 1),
 ('Film-Noir', 1)]

In [51]:
preds_ials[preds_ials[USER_COL] == 0].merge(movies, on=ITEM_COL)

,user_id,item_id,rating,title,genre
0,0,900,0.874526,Alien (1979),"[Action, Horror, Sci-Fi, Thriller]"
1,0,886,0.853599,Aliens (1986),"[Action, Sci-Fi, Thriller, War]"
2,0,954,0.776213,Back to the Future (1985),"[Comedy, Sci-Fi]"
3,0,1175,0.7754,Men in Black (1997),"[Action, Adventure, Comedy, Sci-Fi]"
4,0,2701,0.727444,Predator (1987),"[Action, Sci-Fi, Thriller]"
5,0,2237,0.726934,Total Recall (1990),"[Action, Adventure, Sci-Fi, Thriller]"
6,0,2027,0.721783,"Rocky Horror Picture Show, The (1975)","[Comedy, Horror, Musical, Sci-Fi]"
7,0,1045,0.681578,Star Trek IV: The Voyage Home (1986),"[Action, Adventure, Sci-Fi]"
8,0,1866,0.656831,"Fly, The (1958)","[Horror, Sci-Fi]"
9,0,2189,0.62761,American Beauty (1999),"[Comedy, Drama]"
